In [1]:
import h5py
from shutil import copyfile
import tempfile

# I am importing pycroscopy from the original package but you will have to change yours
import sys
sys.path.append('/Users/syz/PycharmProjects/pyUSID/')
sys.path.append('/Users/syz/PycharmProjects/pycroscopy/')
from pycroscopy.processing import SignalFilter
from pycroscopy.processing import fft
# You will need to do something like:
#from fft import LowPassFilter
#from mpi_signal_filter import SignalFilter

In [2]:
orig_path = 'GLine_8V_10kHz_256x256_0001_clean.h5'

with tempfile.TemporaryDirectory() as tmp_dir:
    h5_path = tmp_dir + 'gline.h5'
    copyfile(orig_path, h5_path)

In [2]:
h5_path = 'GLine_8V_10kHz_256x256_0001_clean.h5'

In [3]:
h5_f = h5py.File(h5_path, mode='r+')
# Not necessary I think but Chris used it
h5_f.atomic = True # This doesn't seem to make any difference

h5_grp = h5_f['Measurement_000/Channel_000']
h5_main = h5_grp['Raw_Data']

# Adding some important metadata that was missing in the HDF5 file

In [6]:
h5_main.attrs.update({'quantity': 'Deflection', 'units': 'V'})
for dset_name in ['Position_Indices', 'Position_Values']:
    h5_dset = h5_grp[dset_name]
    usid.hdf_utils.write_simple_attrs(h5_dset, {'labels': ['X', 'Y'],
                                                'units': ['au', 'au']})
for dset_name in ['Spectroscopic_Indices', 'Spectroscopic_Values']:
    h5_dset = h5_grp[dset_name]
    usid.hdf_utils.write_simple_attrs(h5_dset, {'labels': ['Bias'],
                                                'units': ['V']})

In [4]:
samp_rate = 4E+6 # I am cheating here
ex_freq = h5_grp.attrs['center frequency [Hz]']
num_spectral_pts = h5_main.shape[1]

frequency_filters = [fft.HarmonicPassFilter(num_spectral_pts, samp_rate, ex_freq, 1E+3, 10)]
noise_tol = 1E-4

sig_filt = SignalFilter(h5_main, frequency_filters=frequency_filters,
                           noise_threshold=noise_tol, write_filtered=True,
                           write_condensed=False, num_pix=1, verbose=True)

/Users/syz/PycharmProjects/pyUSID/pyUSID/io/usi_data.py:148: UserWarning: This dataset does not have an N-dimensional form
  warn('This dataset does not have an N-dimensional form')


Consider calling test() to check results before calling compute() which computes on the entire dataset and writes back to the HDF5 file
No mpi4py found or script was not called via mpixexec / mpirun. Assuming single node computation
Rank 0 - on socket with 4 logical cores and 6.39 GB avail. RAM shared by 1 ranks each given 3 cores.
Allowed to read 43690 pixels per chunk
Allowed to read 14563 pixels per chunk
Checking for duplicates:


In [17]:
h5_filt_grp = sig_filt.compute()

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'

In [8]:


h5_f.close()